In [1]:
import csv
import string
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
#data import

with open('history.txt') as f:
  content = f.read()

content = " ".join(content.split())
content = content.strip()
content = content.lower()

content = content[100000:400000]

print(len(content))

300000


In [3]:
# tokenize to words and sentenses (word sequences)

from keras.preprocessing.text import text_to_word_sequence
words = text_to_word_sequence(content)

print(words)
print(len(words))
elems = content.split('.')
print(elems)

['ith', 'them', 'all', 'or', 'most', 'of', 'a', 'certain', 'group', 'of', 'curious', 'ideas', 'and', 'practices', 'some', 'of', 'them', 'are', 'such', 'queer', 'ideas', 'that', 'they', 'call', 'for', 'the', 'explanation', 'of', 'the', 'mental', 'expert', 'they', 'made', 'pyramids', '70', 'and', 'great', 'mounds', 'and', 'set', 'up', 'great', 'circles', 'of', 'big', 'stones', 'perhaps', 'to', 'facilitate', 'the', 'astronomical', 'observation', 'of', 'the', 'priests', 'they', 'made', 'mummies', 'of', 'some', 'or', 'all', 'of', 'their', 'dead', 'they', 'tattooed', 'and', 'circumcized', 'they', 'had', 'the', 'old', 'custom', 'known', 'as', 'the', 'couvade', 'of', 'sending', 'the', 'father', 'to', 'bed', 'and', 'rest', 'when', 'a', 'child', 'was', 'born', 'and', 'they', 'had', 'as', 'a', 'luck', 'symbol', 'the', 'well', 'known', 'swastika', 'if', 'we', 'were', 'to', 'make', 'a', 'map', 'of', 'the', 'world', 'with', 'dots', 'to', 'show', 'how', 'far', 'these', 'group', 'practices', 'have', '

In [4]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(elems)
word_index = tokenizer.word_index
print(word_index)
max_words = len(word_index)
print(max_words)

{'the': 1, 'of': 2, 'and': 3, 'in': 4, 'a': 5, 'to': 6, 'was': 7, 'were': 8, 'that': 9, 'it': 10, 'had': 11, 'he': 12, 'they': 13, 'his': 14, 'for': 15, 'but': 16, 'this': 17, 'as': 18, 'by': 19, 'at': 20, 'with': 21, 'from': 22, 'their': 23, 'great': 24, 'there': 25, 'is': 26, 'not': 27, 'all': 28, 'or': 29, 'an': 30, 'world': 31, 'these': 32, 'empire': 33, 'have': 34, 'on': 35, 'which': 36, 'b': 37, 'c': 38, 'into': 39, 'roman': 40, 'people': 41, 'who': 42, 'very': 43, 'its': 44, 'first': 45, 'upon': 46, 'we': 47, 'no': 48, 'been': 49, 'one': 50, 'time': 51, 'life': 52, 'more': 53, 'him': 54, 'be': 55, 'them': 56, 'made': 57, 'new': 58, 'egypt': 59, 'century': 60, 'so': 61, 'illustration': 62, 'up': 63, 'rome': 64, 'now': 65, 'history': 66, 'under': 67, 'greek': 68, 'god': 69, 'asia': 70, 'men': 71, 'came': 72, 'before': 73, 'over': 74, 'north': 75, 'still': 76, 'peoples': 77, 'only': 78, 'about': 79, 'would': 80, 'power': 81, 'years': 82, 'after': 83, 'than': 84, 'city': 85, 'most':

In [5]:
arr = []

for w in words:
  arr.append(word_index[w])

x = np.array(arr)
print(x.shape)

(51045,)


In [6]:
# sliding window
from sklearn.model_selection import train_test_split
window_size = 10

in_data = np.zeros((len(x)-window_size, window_size))
out_datax = np.zeros((len(x)-window_size,1))

for i in range(len(x)-window_size):
  in_data[i]=x[i:i+window_size]
  out_datax[i]=x[i+window_size:i+window_size+1]

out_data=tf.keras.utils.to_categorical(out_datax,num_classes=max_words+1)
print(out_data.shape)
print(in_data,out_datax)

print(in_data.shape, out_data.shape)
print(in_data, out_data)


x_train, x_test, y_train, y_test = train_test_split(in_data, out_data, test_size=0.2, random_state=123)

del in_data
del out_data

(51035, 6685)
[[3.304e+03 5.600e+01 2.800e+01 ... 2.840e+02 4.880e+02 2.000e+00]
 [5.600e+01 2.800e+01 2.900e+01 ... 4.880e+02 2.000e+00 1.315e+03]
 [2.800e+01 2.900e+01 8.600e+01 ... 2.000e+00 1.315e+03 1.560e+02]
 ...
 [2.000e+00 1.000e+00 1.260e+02 ... 5.190e+02 7.000e+00 1.000e+00]
 [1.000e+00 1.260e+02 2.163e+03 ... 7.000e+00 1.000e+00 1.832e+03]
 [1.260e+02 2.163e+03 6.684e+03 ... 1.000e+00 1.832e+03 2.000e+00]] [[1.315e+03]
 [1.560e+02]
 [3.000e+00]
 ...
 [1.832e+03]
 [2.000e+00]
 [2.933e+03]]
(51035, 10) (51035, 6685)
[[3.304e+03 5.600e+01 2.800e+01 ... 2.840e+02 4.880e+02 2.000e+00]
 [5.600e+01 2.800e+01 2.900e+01 ... 4.880e+02 2.000e+00 1.315e+03]
 [2.800e+01 2.900e+01 8.600e+01 ... 2.000e+00 1.315e+03 1.560e+02]
 ...
 [2.000e+00 1.000e+00 1.260e+02 ... 5.190e+02 7.000e+00 1.000e+00]
 [1.000e+00 1.260e+02 2.163e+03 ... 7.000e+00 1.000e+00 1.832e+03]
 [1.260e+02 2.163e+03 6.684e+03 ... 1.000e+00 1.832e+03 2.000e+00]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0.

In [7]:
#Glove
max_embed = 50
path = 'glove.6B.50d.txt'
embeddings_index = {}

with open(path) as f:
  for line in f:
    values = line.split()
    word = values[0]
    coeffs = np.array(values[1:max_embed+1], dtype = 'float32')
    embeddings_index[word] = coeffs

dict(list(embeddings_index.items())[0:2])

{'the': array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
       dtype=float32),
 ',': array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
        -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
        -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
        -0.4

In [8]:
# Create matrix of embeddings for our words

print(word_index)
embeddings_matrix = np.zeros((max_words+1, max_embed))
for word, i in word_index.items():
    embeddings_vector = embeddings_index.get(word)
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
print(embeddings_matrix.shape)
del embeddings_index


{'the': 1, 'of': 2, 'and': 3, 'in': 4, 'a': 5, 'to': 6, 'was': 7, 'were': 8, 'that': 9, 'it': 10, 'had': 11, 'he': 12, 'they': 13, 'his': 14, 'for': 15, 'but': 16, 'this': 17, 'as': 18, 'by': 19, 'at': 20, 'with': 21, 'from': 22, 'their': 23, 'great': 24, 'there': 25, 'is': 26, 'not': 27, 'all': 28, 'or': 29, 'an': 30, 'world': 31, 'these': 32, 'empire': 33, 'have': 34, 'on': 35, 'which': 36, 'b': 37, 'c': 38, 'into': 39, 'roman': 40, 'people': 41, 'who': 42, 'very': 43, 'its': 44, 'first': 45, 'upon': 46, 'we': 47, 'no': 48, 'been': 49, 'one': 50, 'time': 51, 'life': 52, 'more': 53, 'him': 54, 'be': 55, 'them': 56, 'made': 57, 'new': 58, 'egypt': 59, 'century': 60, 'so': 61, 'illustration': 62, 'up': 63, 'rome': 64, 'now': 65, 'history': 66, 'under': 67, 'greek': 68, 'god': 69, 'asia': 70, 'men': 71, 'came': 72, 'before': 73, 'over': 74, 'north': 75, 'still': 76, 'peoples': 77, 'only': 78, 'about': 79, 'would': 80, 'power': 81, 'years': 82, 'after': 83, 'than': 84, 'city': 85, 'most':

In [9]:
from keras import backend as K

import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Embedding (input_dim= max_words+1, output_dim=max_embed, weights=[embeddings_matrix], input_length=window_size, trainable=False),
  tf.keras.layers.SimpleRNN(256, return_sequences=True),
  tf.keras.layers.Dropout (0.2), tf.keras.layers.SimpleRNN(256),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(max_words+1, activation='softmax')])

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

2022-11-24 09:26:18.584173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 50)            334250    
                                                                 
 simple_rnn (SimpleRNN)      (None, 10, 256)           78592     
                                                                 
 dropout (Dropout)           (None, 10, 256)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 6685)              1

In [10]:
# training

history = model.fit(x_train, y_train, epochs = 10, validation_split=0.2, verbose = 1, batch_size = 32)

Epoch 1/10
1021/1021 [==============================] - 85s 79ms/step - loss: 6.7484 - accuracy: 0.0810 - val_loss: 6.6973 - val_accuracy: 0.0841
Epoch 2/10
1021/1021 [==============================] - 76s 74ms/step - loss: 6.5480 - accuracy: 0.0834 - val_loss: 6.7553 - val_accuracy: 0.0841
Epoch 3/10
1021/1021 [==============================] - 81s 79ms/step - loss: 6.4669 - accuracy: 0.0925 - val_loss: 6.8795 - val_accuracy: 0.0999
Epoch 4/10
1021/1021 [==============================] - 75s 73ms/step - loss: 6.3842 - accuracy: 0.0992 - val_loss: 6.7830 - val_accuracy: 0.1027
Epoch 5/10
1021/1021 [==============================] - 77s 75ms/step - loss: 6.3212 - accuracy: 0.1013 - val_loss: 6.8252 - val_accuracy: 0.0972
Epoch 6/10
1021/1021 [==============================] - 76s 74ms/step - loss: 6.2908 - accuracy: 0.1014 - val_loss: 6.8749 - val_accuracy: 0.1007
Epoch 7/10
1021/1021 [==============================] - 97s 95ms/step - loss: 6.2505 - accuracy: 0.1027 - val_loss: 6.8778 -

In [11]:
def prediction(in_phase):
  in_phase=in_phase.strip()
  in_phase=in_phase.lower()
  print(in_phase)
    
  in_words=text_to_word_sequence(in_phase)
    
#replace words with indexes
  arr=[]
  for w in in_words[:window_size]:
    arr.append(word_index[w])
  x=np.array(arr)

  if len(x)<window_size:
    z=np.zeros((window_size-len(x)))
    x=np.concatenate((z,x))

  x=x.reshape(1,len(x))
  p=model.predict(x)
  predicted=np.argmax(p,axis=-1)
  key=[k for k,v in word_index.items() if v== predicted]
  return key

in_text="I understand that they could see"
for i in range(10):
    pr=prediction(in_text)
    print(pr)
    in_text=in_text+" "+pr[0]

i understand that they could see
1/1 [==============================] - 1s 818ms/step
['the']
i understand that they could see the
1/1 [==============================] - 0s 36ms/step
['the']
i understand that they could see the the
1/1 [==============================] - 0s 42ms/step
['the']
i understand that they could see the the the
1/1 [==============================] - 0s 49ms/step
['the']
i understand that they could see the the the the
1/1 [==============================] - 1s 514ms/step
['the']
i understand that they could see the the the the the
1/1 [==============================] - 0s 29ms/step
['the']
i understand that they could see the the the the the the
1/1 [==============================] - 0s 28ms/step
['the']
i understand that they could see the the the the the the the
1/1 [==============================] - 0s 30ms/step
['the']
i understand that they could see the the the the the the the the
1/1 [==============================] - 0s 26ms/step
['the']
i understand that